# Análise dos preços de apartamentos em Salvador

A análise será feita com os dados extraídos na OLX em 04/02/2023.

### Extração dos dados no site da OLX

In [175]:
import pandas as pd
import numpy as np
import datetime as dt
import requests
from bs4 import BeautifulSoup

page_olx = 1
URL = "https://www.olx.com.br/imoveis/venda/estado-ba/grande-salvador/salvador?o={page_olx}"

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36',
    'accept-language':'pt-BR,pt;q=0.6',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding':'gzip, deflate'
}

response = requests.get(url=URL,headers=headers)
salvador_olx_data = response.text
soup = BeautifulSoup(salvador_olx_data, "html.parser")

In [ ]:
#Lista de itens na página
page_data_items = soup.select("#ad-list > li")
item = page_data_items[10]

#Título do anuncio:
titulo = item.select("a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-gpHHfC.kJrkKt > h2")
titulo_anuncio = titulo[0].text

#Tipo de imóvel:
tp_imovel_item = item.select("h3")
tp_imovel_item = [x for x in tp_imovel_item if "venda" in x.text]
tp_imovel = tp_imovel_item[0].text

#Informações que não existem em todos os anuncios
quarto_info, area_info, garagem_info, banheiro_info, iptu_info, condominio_info = '','','','','',''

#Informações de banheiro, area, quartos e garagem:
for i in range(1,8):
    try:
        atributo = item.select(f"ul > li:nth-child({i}) > span")[0].attrs['aria-label']
        if 'QUARTO' in atributo.upper():
            quarto_info = atributo
        elif 'METRO' in atributo.upper():
            area_info = atributo
        elif 'GARAGEM' in atributo.upper():
            garagem_info = atributo
        elif 'BANHEIRO' in atributo.upper():
            banheiro_info = atributo 
    except:
        pass

#Dados de localização:
localizacao_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(2)')[0]
cidade, bairro = [x.strip() for x in localizacao_item.text.split(',')]

#Valores:
preco_item = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span.main-price')
preco_imovel = preco_item[0].text

#Condomínio e IPTU:
for i in range(1,4):
    try:
        valor_atributo = item.select(f'a > div.sc-fgfRvd.fZoIzL > div.sc-hIVACf.dUnJVd > div.sc-cpmKsF.ecSOri > span:nth-child({i})')[0].text
        if 'IPTU' in valor_atributo.upper():
            iptu_info = valor_atributo
        elif 'CONDOM' in valor_atributo.upper():
            condominio_info = valor_atributo
    except:
        pass

#Link do anúncio:
link_anuncio = item.select('a')[0].attrs['href']
id_anuncio = link_anuncio.split('-')[-1]


#Data do anúncio:
data_anuncio_info = item.select('a > div.sc-fgfRvd.fZoIzL > div.sc-kQsIoO.kbWREw > div > div.sc-hjRWVT.lgwjMd > span:nth-child(4)')[0].text


In [ ]:
print(f'{titulo_anuncio}\n{tp_imovel}\n{quarto_info}\n{area_info}\n{garagem_info}\n{banheiro_info}\n{cidade}\n{bairro}\n{preco_imovel}\n{iptu_info}\n{condominio_info}\n{link_anuncio}\n{data_anuncio_info}')


VENDO APARTAMENTO - 59 M² - 1 QUARTOS - CELEBRATION GARIBALDI  RIO VERMELHOR - SALVADOR | 
Apartamento À venda
1 quarto
59 metros quadrados
1 vaga de garagem
1 banheiro
Salvador
Rio Vermelho
R$ 650.000
IPTU R$ 101
Condomínio R$ 423
https://ba.olx.com.br/grande-salvador/imoveis/vendo-apartamento-59-m-1-quartos-celebration-garibaldi-rio-vermelhor-salvador-1113267526
Hoje, 08:12


In [ ]:
#Convertendo os dados quantitativos para string:
def string_to_float_olx(string):
    try:
        return float(string.split()[0])
    except (ValueError, IndexError) as e:
            try:
                return float(string.split()[-1])
            except (ValueError, IndexError) as e:
                return np.nan
            
qtd_quartos = string_to_float_olx(quarto_info)
area_m2 = string_to_float_olx(area_info)
vagas_garagem = string_to_float_olx(garagem_info)
qtd_banheiros = string_to_float_olx(banheiro_info)
valor_imovel = string_to_float_olx(preco_imovel.replace('.',''))
valor_iptu = string_to_float_olx(iptu_info.replace('.',''))
valor_condominio = string_to_float_olx(condominio_info.replace('.',''))

In [ ]:
# Função para converter as datas dos anúncios:
def olx_date_to_datetime(date_string):
    if 'HOJE' in date_string.upper():
        date_anuncio = dt.datetime.now()
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1]))
    elif 'ONTEM' in date_string.upper():
        date_anuncio = dt.datetime.now() + dt.timedelta(days=-1)
        return dt.datetime(date_anuncio.year, date_anuncio.month, date_anuncio.day, int(date_string.split(', ')[1].split(':')[0]), int(date_string.split(', ')[1].split(':')[1])) 
    try:
        hoje = dt.datetime.now()
        day_month, hour_min = date_string.split(', ')
        month_map = {"jan": 1,"fev": 2,"mar": 3,"abr": 4,"mai": 5,"jun": 6,"jul": 7,"ago": 8,"set": 9,"out": 10,"nov": 11,"dez": 12}
        mes = month_map[day_month.split()[-1].lower()]
        dia = int(day_month.split()[0])
        hora, minuto = map(int, hour_min.split(":"))
        candidate_date = dt.datetime(hoje.year, mes, dia, hora, minuto)

        if candidate_date > hoje:
            ano = hoje.year - 1
        else:
            ano = hoje.year
        return dt.datetime(ano, mes, dia, hora, minuto)
    except (ValueError, KeyError):
        return None

#Teste função
# data1 = 'Hoje, 08:12'
# data2 = '3 de fev, 16:18'
# data3 = 'Ontem, 23:58'

# olx_date_to_datetime(data1), olx_date_to_datetime(data2), olx_date_to_datetime(data3)

data_anuncio_datetime = olx_date_to_datetime(data_anuncio_info).strftime('%Y-%m-%d %H:%M')


In [ ]:
#Criação do Dataframe:
anuncio_dict = {
    'ID_ANUNCIO': id_anuncio,
    'TIPO_IMOVEL': tp_imovel,
    'TITULO': titulo_anuncio,
    'QTD_QUARTOS': qtd_quartos,
    'AREA_M2': area_m2,
    'VAGAS_GARAGEM': vagas_garagem,
    'QTD_BANHEIROS': qtd_banheiros,
    'CIDADDE': cidade,
    'BAIRRO': bairro,
    'VALOR_RS': valor_imovel,
    'IPTU_RS': valor_iptu,
    'CONDOMINIO_RS': valor_condominio,
    'DATA_PUBLICACAO': data_anuncio_datetime,
    'LINK': link_anuncio,
}

df_anuncio_id = pd.DataFrame(data=anuncio_dict.items()).T
df_anuncio_id.columns = df_anuncio_id.iloc[0]
df_anuncio_id = df_anuncio_id[1:]

#Agrupar anuncios em data frame TT:
try:
    df_anuncios_tt = pd.concat([df_anuncios_tt, df_anuncio_id])
except:
    df_anuncios_tt = df_anuncio_id

In [ ]:
df_anuncios_tt


,ID_ANUNCIO,TIPO_IMOVEL,TITULO,QTD_QUARTOS,AREA_M2,VAGAS_GARAGEM,QTD_BANHEIROS,CIDADDE,BAIRRO,VALOR_RS,IPTU_RS,CONDOMINIO_RS,DATA_PUBLICACAO,LINK
1,1113267526,Apartamento À venda,VENDO APARTAMENTO - 59 M² - 1 QUARTOS - CELEBR...,1.0,59.0,1.0,1.0,Salvador,Rio Vermelho,650000.0,101.0,423.0,2023-02-05 08:12,https://ba.olx.com.br/grande-salvador/imoveis/...
